In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=ebb8252c049e20886ea9492ed16b0ee11e8c0d954a0efe1e99f6d44f144a3fdc
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
#Import libraries
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [3]:
#Initialize Spark Session
spark = SparkSession.builder \
    .appName("YassirTaxiService") \
    .getOrCreate()

In [5]:
source_data = spark.read.csv("test.csv", header=True)

# Get the first line of the DataFrame.
first_line = source_data.head()

# Display the first line.
print(first_line)

Row(ID='0x2318 ', Delivery_person_ID='COIMBRES13DEL01 ', Delivery_person_Age='NaN ', Delivery_person_Ratings='NaN ', Restaurant_latitude='11.003669', Restaurant_longitude='76.976494', Delivery_location_latitude='11.043669', Delivery_location_longitude='77.016494', Order_Date='30-03-2022', Time_Orderd='NaN ', Time_Order_picked='15:05:00', Weatherconditions='conditions NaN', Road_traffic_density='NaN ', Vehicle_condition='3', Type_of_order='Drinks ', Type_of_vehicle='electric_scooter ', multiple_deliveries='1', Festival='No ', City='Metropolitian ')


In [6]:
#print schema
source_data.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Delivery_person_ID: string (nullable = true)
 |-- Delivery_person_Age: string (nullable = true)
 |-- Delivery_person_Ratings: string (nullable = true)
 |-- Restaurant_latitude: string (nullable = true)
 |-- Restaurant_longitude: string (nullable = true)
 |-- Delivery_location_latitude: string (nullable = true)
 |-- Delivery_location_longitude: string (nullable = true)
 |-- Order_Date: string (nullable = true)
 |-- Time_Orderd: string (nullable = true)
 |-- Time_Order_picked: string (nullable = true)
 |-- Weatherconditions: string (nullable = true)
 |-- Road_traffic_density: string (nullable = true)
 |-- Vehicle_condition: string (nullable = true)
 |-- Type_of_order: string (nullable = true)
 |-- Type_of_vehicle: string (nullable = true)
 |-- multiple_deliveries: string (nullable = true)
 |-- Festival: string (nullable = true)
 |-- City: string (nullable = true)



In [7]:
#Show data
source_data.show()

+-------+------------------+-------------------+-----------------------+-------------------+--------------------+--------------------------+---------------------------+----------+-----------+-----------------+-----------------+--------------------+-----------------+-------------+-----------------+-------------------+--------+--------------+
|     ID|Delivery_person_ID|Delivery_person_Age|Delivery_person_Ratings|Restaurant_latitude|Restaurant_longitude|Delivery_location_latitude|Delivery_location_longitude|Order_Date|Time_Orderd|Time_Order_picked|Weatherconditions|Road_traffic_density|Vehicle_condition|Type_of_order|  Type_of_vehicle|multiple_deliveries|Festival|          City|
+-------+------------------+-------------------+-----------------------+-------------------+--------------------+--------------------------+---------------------------+----------+-----------+-----------------+-----------------+--------------------+-----------------+-------------+-----------------+----------------

In [12]:
#Create a temporary view for the DataFrame
source_data.createOrReplaceTempView("taxi_service")

In [39]:
#Number of deliveries made by a given delivery person.
driver_id = "COIMBRES13DEL01"
num_deliveries_by_driver = spark.sql(f"""
    SELECT COUNT(*) AS num_deliveries
    FROM taxi_service
    WHERE Delivery_person_ID like '%{driver_id}%'
""").collect()[0][0]
print("Nombre de livraisons effectuées par le livreur", driver_id, ":", num_deliveries_by_driver)

Nombre de livraisons effectuées par le livreur COIMBRES13DEL01 : 23


In [42]:
#Total revenue generated by a delivery person over a given period.
start_date = "01-01-2022"
end_date = "31-03-2023"
driver_id = "COIMBRES13DEL01"
total_revenue_by_driver = spark.sql(f"""
    SELECT SUM(CASE WHEN Type_of_order = 'Buffet ' THEN 10
                    WHEN Type_of_order = 'Snack ' THEN 15
                    ELSE 0 END) AS total_revenue
    FROM taxi_service
    WHERE Delivery_person_ID like '%{driver_id}%'
    AND Order_Date BETWEEN '{start_date}' AND '{end_date}'
""").collect()[0][0]
print("Revenu total généré par le livreur", driver_id, "entre", start_date, "et", end_date, ":", total_revenue_by_driver)

Revenu total généré par le livreur COIMBRES13DEL01 entre 01-01-2022 et 31-03-2023 : 155


In [45]:
#Top 10 highest-rated delivery persons.
top_rated_drivers = spark.sql("""
    SELECT Delivery_person_ID, AVG(Delivery_person_Ratings) AS avg_rating
    FROM taxi_service WHERE  Delivery_person_Ratings !='NaN '
    GROUP BY Delivery_person_ID
    ORDER BY avg_rating DESC
    LIMIT 10
""")
top_rated_drivers.show()

+------------------+-----------------+
|Delivery_person_ID|       avg_rating|
+------------------+-----------------+
|    KNPRES14DEL02 |5.066666666666666|
|   AURGRES16DEL03 |              5.0|
|   AGRRES010DEL02 |              5.0|
|   AURGRES08DEL01 |              5.0|
|    KOCRES13DEL03 |              5.0|
|    KOLRES03DEL01 |              5.0|
|    KOCRES17DEL03 |              5.0|
|    BHPRES15DEL02 |              5.0|
|    KOCRES07DEL01 |              5.0|
|    DEHRES04DEL03 |              5.0|
+------------------+-----------------+



In [47]:
#Total number of deliveries per order type.
deliveries_by_order_type = spark.sql("""
    SELECT Type_of_order, COUNT(*) AS num_deliveries
    FROM taxi_service
    GROUP BY Type_of_order
""")
deliveries_by_order_type.show()


+-------------+--------------+
|Type_of_order|num_deliveries|
+-------------+--------------+
|       Snack |          2815|
|      Buffet |          2870|
|      Drinks |          2920|
|        Meal |          2794|
+-------------+--------------+



In [48]:
#Percentage of multiple deliveries relative to the total deliveries
multiple_deliveries_percentage = spark.sql("""
    SELECT
        (COUNT(CASE WHEN multiple_deliveries = True THEN 1 END) / COUNT(*)) * 100 AS multiple_deliveries_percentage
    FROM taxi_service
""").collect()[0][0]
print("Pourcentage de livraisons multiples :", multiple_deliveries_percentage)

Pourcentage de livraisons multiples : 61.97912097552417
